# Vérification de signature d'emails avec Python
Dans cette Tech Review, nous présentons les différentes solutions étudiées pour vérifier des
emails signés avec Python.  

## Import des librairies


In [2]:
import email
import subprocess
from pprint import pprint

from asn1crypto import cms
from cryptography.hazmat.bindings._rust import test_support
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.asymmetric.ec import ECDSA, EllipticCurvePublicKey
from cryptography.hazmat.primitives.asymmetric.rsa import RSAPublicKey
from cryptography.hazmat.primitives.serialization import Encoding, load_pem_private_key
from cryptography.hazmat.primitives.serialization.pkcs7 import (
    PKCS7Options,
    PKCS7SignatureBuilder,
)
from cryptography.x509 import load_pem_x509_certificate
from cryptography.x509.verification import PolicyBuilder, Store
from endesive import email as endesive_email

## Lecture du certificat X.509

In [3]:
# Clé publique : certificat RSA
with open("vectors/verify/ca.pem", "rb") as file:
    certificate_bytes = file.read()
    
certificate = load_pem_x509_certificate(certificate_bytes)

cryptography.hazmat.bindings._rust.x509.Certificate

In [4]:
# Clé publique du certificat
public_key = certificate.public_key()
print("Type:", type(public_key))

Type: <class 'cryptography.hazmat.bindings._rust.openssl.rsa.RSAPublicKey'>


In [5]:
# Chaîne de certification utilisée
# Clé publique : certificat RSA
with open("vectors/verify/ca_chain.pem", "rb") as file:
    certificate_chain_bytes = file.read()
    
certificate_chain = load_pem_x509_certificate(certificate_chain_bytes)

In [6]:
# Clé privée : RSA
with open("vectors/verify/ca_key.pem", "rb") as file:
    private_key = load_pem_private_key(file.read(), password=None)
print("Type:", type(private_key))

Type: <class 'cryptography.hazmat.bindings._rust.openssl.rsa.RSAPrivateKey'>


## Signature d'un email

In [7]:
# Écriture d'un message non-chiffré
# message = b"Hello world!\n"
message = b"Hello world!"
canonical_message = b"Content-Type: text/plain\r\n\r\n" + message
with open("vectors/message.txt", "wb") as file:
    file.write(message)
with open("vectors/canonical_message.txt", "wb") as file:
    file.write(canonical_message)

### Avec `openssl`

In [11]:
# SMIME : ancienne version
instructions = [
    "openssl",
    "smime",  # on peut utiliser smime ou cms
    "-sign",
    "-in",
    "vectors/message.txt",
    "-out",
    "vectors/verify/signed.txt",
    "-signer",
    "vectors/verify/ca.pem",
    "-inkey",
    "vectors/verify/ca_key.pem",
    # "-md",  # optionnel
    # "-sha512",  # optionnel
    # "-noattr",  # optionnel
    # "-text",  # optionnel
]
smime_encryption_result = subprocess.run(instructions, capture_output=True)
smime_encryption_result.stderr

b''

In [14]:
# Signature
instructions = [
    "openssl",
    "smime",  # on peut utiliser smime ou cms
    "-sign",
    "-in",
    "vectors/message.txt",
    "-out",
    "vectors/verify/signature.der",
    "-outform",
    "DER",
    "-signer",
    "vectors/verify/ca.pem",
    "-inkey",
    "vectors/verify/ca_key.pem",
    # "-md",  # optionnel
    # "-sha512",  # optionnel
    # "-noattr",  # optionnel
    # "-text",  # optionnel
]
smime_encryption_result = subprocess.run(instructions, capture_output=True)
smime_encryption_result.stderr

b''

In [15]:
with open("vectors/verify/signed.txt", "rb") as file:
    signed = file.read()
with open("vectors/verify/signature.der", "rb") as file:
    signature = file.read()

### Avec `cryptography`
Les fonctionnalités de chiffrage d'email sont disponibles dans la librairie `cryptography` depuis
début 2024. 

In [16]:
# Prepare the envelope builder
builder = PKCS7SignatureBuilder()
builder = builder.add_signer(certificate, private_key, hashes.SHA256())
builder = builder.set_data(message)

# Encrypt the data
options = [
    # PKCS7Options.DetachedSignature,
    # PKCS7Options.NoAttributes,
    # PKCS7Options.Text,
    # PKCS7Options.NoCerts,
]
signed = builder.sign(Encoding.SMIME, options)
signature = builder.sign(Encoding.DER, options)

In [17]:
# Store the data in vectors
with open("vectors/verify/signed.txt", "wb") as file:
    file.write(signed)
with open("vectors/verify/signature.der", "wb") as file:
    file.write(signature)

## Solutions abordées

### Solution 1: avec `openssl`

#### Structure ASN.1

In [18]:
instructions = [
    "openssl",
    "asn1parse",
    "-in",
    "vectors/verify/signature.der",
    "-inform",
    "der",
]
output = subprocess.run(instructions, check=True, capture_output=True)
print(output.stdout.decode())

    0:d=0  hl=4 l=1591 cons: SEQUENCE          
    4:d=1  hl=2 l=   9 prim: OBJECT            :pkcs7-signedData
   15:d=1  hl=4 l=1576 cons: cont [ 0 ]        
   19:d=2  hl=4 l=1572 cons: SEQUENCE          
   23:d=3  hl=2 l=   1 prim: INTEGER           :01
   26:d=3  hl=2 l=  15 cons: SET               
   28:d=4  hl=2 l=  13 cons: SEQUENCE          
   30:d=5  hl=2 l=   9 prim: OBJECT            :sha256
   41:d=5  hl=2 l=   0 prim: NULL              
   43:d=3  hl=2 l=  27 cons: SEQUENCE          
   45:d=4  hl=2 l=   9 prim: OBJECT            :pkcs7-data
   56:d=4  hl=2 l=  14 cons: cont [ 0 ]        
   58:d=5  hl=2 l=  12 prim: OCTET STRING      :Hello world!
   72:d=3  hl=4 l=1019 cons: cont [ 0 ]        
   76:d=4  hl=4 l=1015 cons: SEQUENCE          
   80:d=5  hl=4 l= 735 cons: SEQUENCE          
   84:d=6  hl=2 l=   3 cons: cont [ 0 ]        
   86:d=7  hl=2 l=   1 prim: INTEGER           :02
   89:d=6  hl=2 l=  16 prim: INTEGER           :231300F978724BD3AED2A01CD333F255
 

#### Vérification
Ici, on doit rajouter "-noverify" pour ne pas vérifier la signature du certificat, car sinon on a
l'erreur suivante: "self-signed certificate". Nous n'avons pas cette erreur en utilisant la fonction `test_support.pkcs7_verify` de la librairie `cryptography`.

In [19]:
# Message signé
instructions = [
    "openssl",
    "smime",
    "-verify",
    "-in",
    "vectors/verify/signed.txt",
    # "-certfile",
    # "vectors/verify/verify.pem",
    "-CAfile",
    "vectors/verify/ca_chain.pem",
    # "-noverify",
]
output = subprocess.run(instructions, capture_output=True)
output.stderr.decode()

'Verification successful\r\n'

In [20]:
# Message signé
instructions = [
    "openssl",
    "smime",
    "-verify",
    "-in",
    "vectors/verify/signature.der",
    "-inform",
    "der",
    "-content",
    "vectors/message.txt",  # depends on the text signing options
    # "vectors/canonical_message.txt",  # depends on the text signing options
    # "-certfile",  # optional
    # "vectors/verify/ca.pem",  # optional
    "-CAfile",
    "vectors/verify/ca_chain.pem",
    # "-noverify",
]
output = subprocess.run(instructions, capture_output=True)
output.stderr.decode()

'Verification successful\r\n'

### Solution 2 : avec `asn1crypto`

#### Structure ASN.1

In [21]:
# Load the structure
content_info = cms.ContentInfo.load(signature)
content_type: cms.ContentType = content_info["content_type"]
signed_data: cms.SignedData = content_info["content"]

# # About the content type
content_type.native, content_type.dotted

('signed_data', '1.2.840.113549.1.7.2')

In [22]:
def native_to_json(native_content):
    if isinstance(native_content, list):
        return [native_to_json(item) for item in native_content]
    elif isinstance(native_content, dict):
        return {key: native_to_json(value) for key, value in native_content.items()}
    else:
        return native_content

In [23]:
# About the signed data
# -- Version
version: cms.CMSVersion = signed_data["version"]
print("Version:", version.native)

# -- Digest algorithms
digest_algorithms: cms.DigestAlgorithms = signed_data["digest_algorithms"]
for i, digest_algorithm in enumerate(digest_algorithms):
    print(f"\nDigest algorithm {i+1}:", digest_algorithm["algorithm"].native)

# -- Encapsulated content info
encap_content_info: cms.EncapsulatedContentInfo = signed_data["encap_content_info"]
print("Encapsulated content info:", native_to_json(encap_content_info.native))

# -- Certificates
certificates: cms.CertificateSet = signed_data["certificates"]
for i, cert in enumerate(certificates):
    print(f"\nCertificate {i+1}:")
    pprint(native_to_json(cert.native))

# -- Signer infos
signer_infos: cms.SignerInfos = signed_data["signer_infos"]
for i, signer_info in enumerate(signer_infos):
    print(f"\nSigner info {i+1}:")
    pprint(native_to_json(signer_info.native))

Version: v1

Digest algorithm 1: sha256
Encapsulated content info: {'content_type': 'data', 'content': b'Hello world!'}

Certificate 1:
{'signature_algorithm': {'algorithm': 'sha256_rsa', 'parameters': None},
 'signature_value': b'\xbd\xc8\x8a\xc7\x1a"\xaa\x05\x04\xb3(\xc7w\x08\x84 '
                    b'\rX\xef\xd4\xd5\xe3\xaf\x8c8\xdf3\xfbN\x1e\x7f\xba'
                    b'\x8d\xc8\xbf\xa4\xbf\xd9\x1e\x07Q\xb6\xa0\x82'
                    b'\x89\xa0\x8a\xb4\x91\x03\xfd\xff\xfd4U\n\xa9\xc4\x99\xf7'
                    b'\xeb\xf3[DV,\x9cG\x80\x8b\xc2J2\xe9\xa0\xcd:\\)R'
                    b'\x97\x18\xd5\x9e\xfe\xb6\x1d\xbdM\x0b\x81_\r\xf9\xf3\xc8'
                    b'o\x83d{\xac\x9f?\xccB\xe5\xcd\xc0y\xed\x03\xaf'
                    b'\x89\xc8\x1f\xc7Xu"{h\x9e\xad\x06\x07\xdaa\r\xd5r\t\x9b'
                    b'\x99nHl-\xb8f\x96|\xdc\xff\xc3}\xb8\xbaG\x15\xa4\xc7w'
                    b'\t\xb1\x0ef\xaa\x9fI\x81\x96Ps\x95\x1b\xdd?8\xc4\x9f(\x0f'
                    b'\xc8\xd2\x1

In [24]:
# Zoom sur le seul signer info
signer_info = signed_data["signer_infos"][0]

# Les données à l'intérieur
version: cms.CMSVersion = signer_info["version"]
sid: cms.SignerIdentifier = signer_info["sid"]
digest_algorithm: cms.DigestAlgorithm = signer_info["digest_algorithm"]
signature_algorithm: cms.SignedDigestAlgorithm = signer_info["signature_algorithm"]
signature_data: cms.OctetString = signer_info["signature"]

# Signed attributes
signed_attrs: cms.CMSAttributes = signer_info["signed_attrs"]
for i, attribute in enumerate(signed_attrs):
    print(f"\nSigned attribute {i+1}:")
    print(attribute)
    pprint(native_to_json(attribute.native))


Signed attribute 1:
<asn1crypto.cms.CMSAttribute 1682158966544 b'0\x18\x06\t*\x86H\x86\xf7\r\x01\t\x031\x0b\x06\t*\x86H\x86\xf7\r\x01\x07\x01'>
{'type': 'content_type', 'values': ['data']}

Signed attribute 2:
<asn1crypto.cms.CMSAttribute 1682158966032 b'0\x1c\x06\t*\x86H\x86\xf7\r\x01\t\x051\x0f\x17\r250115101820Z'>
{'type': 'signing_time',
 'values': [datetime.datetime(2025, 1, 15, 10, 18, 20, tzinfo=datetime.timezone.utc)]}

Signed attribute 3:
<asn1crypto.cms.CMSAttribute 1682159686672 b'0/\x06\t*\x86H\x86\xf7\r\x01\t\x041"\x04 \xc0S^K\xe2\xb7\x9f\xfd\x93)\x13\x05Ck\xf8\x891NJ?\xae\xc0^\xcf\xfc\xbb}\xf3\x1a\xd9\xe5\x1a'>
{'type': 'message_digest',
 'values': [b'\xc0S^K\xe2\xb7\x9f\xfd\x93)\x13\x05Ck\xf8\x891NJ?\xae\xc0^\xcf'
            b'\xfc\xbb}\xf3\x1a\xd9\xe5\x1a']}

Signed attribute 4:
<asn1crypto.cms.CMSAttribute 1682135114448 b"06\x06\t*\x86H\x86\xf7\r\x01\t\x0f1)0'0\x0b\x06\t`\x86H\x01e\x03\x04\x01*0\x0b\x06\t`\x86H\x01e\x03\x04\x01\x160\x0b\x06\t`\x86H\x01e\x03\x04\x01\x

#### Vérification de la signature
Il faut vérifier la signature en utilisant le certificat (et pas la clé privée !)

In [26]:
# Données à vérifier
if signed_attrs.native is None:
    # verify_data = message
    verify_data = b"Content-Type: text/plain\r\n\r\n" + message
else:
    verify_data = signed_attrs.dump()
    verify_data = b"\x31" + verify_data[1:]  # 0x31 is the SEQUENCE tag

# Algorithme de hachage / digestion
digest_algorithm_name = signer_info["digest_algorithm"]["algorithm"].native
hash = getattr(hashes, digest_algorithm_name.upper())()

# Vérification de la signature
public_key = certificate.public_key()
if isinstance(public_key, RSAPublicKey):
    public_key.verify(signature_data.native, verify_data, padding.PKCS1v15(), hash)
    print("RSA Signature verified!")
elif isinstance(public_key, EllipticCurvePublicKey):
    public_key.verify(signature_data.native, verify_data, ECDSA(hash))
    print("ECDSA Signature verified!")

RSA Signature verified!


#### Vérification des certificats

In [27]:
builder = PolicyBuilder().store(Store([certificate]))
verifier = builder.build_client_verifier()
verified_client = verifier.verify(certificate, [])
verified_client.chain

[<Certificate(subject=<Name(1.2.840.113549.1.9.1=demo1@trisoft.com.pl)>, ...)>]

### Solution 3 : avec `cryptography` !
Il est possible de vérifier la signature d'un mail avec une fonction rust qui appelle les fonctions
OpenSSL. Cette fonction est privée et n'est donc pas censée être utilisée par les utilisateurs de la
librairie. D'où l'intérêt de construire une API publique ! 

In [31]:
# Récupération du contenu - message.replace fonctionne aussi
signed_message = email.message_from_bytes(signed)
content = signed_message.get_payload()[0].get_payload(decode=True)

# Vérification du message signé
test_support.pkcs7_verify(Encoding.SMIME, signed, content, [certificate_chain], [])

# Vérification de la signature
test_support.pkcs7_verify(Encoding.DER, signature, message, [certificate_chain], [])

### Solution 4 : avec `endesive`
La librairie `endesive` permet de vérifier la signature d'un email. Elle est basée sur les
librairies `asn1crypto`, `certvalidator` et `cryptography`.

In [34]:
endesive_email.verify(signed.decode(), [certificate_bytes])

(True, True, True)